![RAGs](../assets/RAGs.png)

---


### Learning Objective:
By the end of the lesson, students will be able to describe common use case for Retrieval augmented generation (RAGs) and apply to a quesiton and answer system. 
 

### About:  
LLM are trained on large general databases that are typically behind 6-8 months or more. Often we want to apply the power of LLM to our own data, such as private company docs or more recent information (such as news). Retrieval Augmented Generation or RAGs make this possible.  

### Prerequisites:
- Intro to Vector Databases and Queries 



### Contents
1. [Imports](#imports)
1. [Load, Split, Embed, Store](#vec-db)
1. [Retrieve](#retrieve) 
1. [Generate](#generate)

### Activities
1. [Q&A Lab](#lab)
    



### References

- [Chroma](https://www.trychroma.com/)
- [OpenAI Embeddings](https://platform.openai.com/docs/guides/embeddings)
- [RAGs](https://python.langchain.com/docs/use_cases/question_answering/)
---


### Installs
pip install chromadb

<a id='imports'></a>
### Imports

In [1]:
from langchain_openai import ChatOpenAI #openai chatbot
from langchain_core.prompts import ChatPromptTemplate #template for chat prompts
from langchain_core.output_parsers import StrOutputParser #output parser for string output 

# loaders and splitters 
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

#vector store and embeddings 
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

# new! runnables for q&A 
from langchain_core.runnables import RunnablePassthrough

<a id='vec-db'></a>

## Intro to RAGs
LLM are trained on large general databases that are typically behind 6-8 months or more. Often we want to apply the power of LLM to our own data, such as private company docs or more recent information (such as news). Retrieval Augmented Generation, or RAGs, make this possible.  

## Load, Split, Embed and Store
In the Introduction to Vector Databases and Queries lesson, we explored how to load, split, embed and store documents in a vector databases. Additionally, we did a simple similarity query. [source](https://python.langchain.com/docs/modules/data_connection/vectorstores/)


## Retrieve and Generate 
#### For RAGs, we will add two new steps: 
1. [Retrieve](#retrieve): Retrieve is similar to a query. This step pulls documents to share with the LLM in the next step.
1. [Generate](#generate): Where we pass a prompt and documentation to the LLM 

Notice that this is similar to how Github Copilot works. 

<a id='load-docs'></a>
#### Load, Split, Embed and Store Our Docs 
We will load and process text from the Wikipedia [Ancient Rome article](https://en.wikipedia.org/wiki/Ancient_Rome) storing its embeddings in a vector database.

In [2]:
# load 
loader = WebBaseLoader("https://en.wikipedia.org/wiki/Ancient_Rome")
pages = loader.load()

#split
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)
split_docs = text_splitter.split_documents(pages)

# embed and store
db = Chroma.from_documents(split_docs, OpenAIEmbeddings(openai_api_key="...", model="text-embedding-3-large"))


<a id='retrieve'></a>
### Retrieve



In [3]:
# Retrieve docs using a similarity search query and save as retrieved_docs
retriever = db.as_retriever(search_type="similarity")
retrieved_docs = retriever.invoke("What structures were built in ancient Rome?")

In [4]:
# explore results 
print("Number of retrieved docs:", len(retrieved_docs))
print(retrieved_docs[0].page_content) # first doc content 
print(retrieved_docs[-1].page_content) # last doc content 

Number of retrieved docs: 4
Culture[edit]
Main article: Culture of ancient Rome
The seven hills of Rome
Life in ancient Rome revolved around the city of Rome, located on seven hills. The city had a vast number of monumental structures like the Colosseum, the Trajan's Forum and the Pantheon. It had theatres, gymnasiums, marketplaces, functional sewers, bath complexes complete with libraries and shops, and fountains with fresh drinking water supplied by hundreds of miles of aqueducts. Throughout the territory under the control of ancient Rome, residential architecture ranged from modest houses to country villas.
The Romans were renowned for their architecture, which is grouped with Greek traditions into "Classical architecture". Although there were many differences from Greek architecture, Rome borrowed heavily from Greece in adhering to strict, formulaic building designs and proportions. Aside from two new orders of columns, composite and Tuscan, and from the dome, which was derived fro

<a id='generate'></a>
### Generate

Now we will take our retrieved documents and combine them with a user prompt and pass all of that to a language model. Here we will build a standard prompt chain with a few modifications to also combine and pass our documents to the model as well. 

In [7]:
# Specify Model 
model = ChatOpenAI(openai_api_key="...", model="gpt-4-0125-preview")

# Prompt
prompt_txt= """Answer the question based only on the following context: {context}
                Question: {question}
            """
prompt = ChatPromptTemplate.from_template(prompt_txt)

# Create RAG Chain 
rag_chain = ( {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)


In [8]:
# Invoke
rag_chain.invoke("What was built in ancient Rome?")

'In ancient Rome, a wide array of structures and engineering marvels were constructed. These included:\n\n1. Roads\n2. Bridges\n3. Aqueducts\n4. Public baths\n5. Theatres\n6. Arenas\n7. Monuments such as the Colosseum, Pont du Gard, and the Pantheon\n8. Sewer systems, notably the Cloaca Maxima\n9. Residential architecture ranging from modest houses to country villas\n10. Functional sewers, bath complexes complete with libraries and shops\n11. Fountains with fresh drinking water supplied by aqueducts'

### You can look under the hood a bit with debug

#### Example to show how it is passed to our language model: 
``` 
"prompts": [
    "Human: Answer the question based only on the following context: 
    
[Document(page_content=\"Culture[edit]\\nMain article: Culture of ancient Rome\\nThe seven hills of Rome\\nLife in ancient Rome revolved around the city of Rome, located on seven hills. The city had a vast number of monumental structures like the Colosseum, the Trajan's Forum and the Pantheon. It had theatres, gymnasiums, marketplaces, functional sewers, bath complexes complete with libraries and shops, and fountains with fresh drinking water supplied by hundreds of miles of aqueducts. Throughout the territory under the control of ancient Rome, residential architecture ranged from modest houses to country villas.\", metadata={'language': 'en', 'source': 'https://en.wikipedia.org/wiki/Ancient_Rome', 'title': 'Ancient Rome - Wikipedia'}), Document(page_content='The Romans were renowned for their architecture, which is grouped with Greek traditions into \"Classical architecture\". Although there were many differences from Greek architecture, Rome borrowed heavily from Greece in adhering to strict, formulaic building designs and proportions. Aside from two new orders of columns, composite and Tuscan, and from the dome, which was derived from the Etruscan arch, Rome had relatively few architectural innovations until the end of the Republic.', metadata={'language': 'en', 'source': 'https://en.wikipedia.org/wiki/Ancient_Rome', 'title': 'Ancient Rome - Wikipedia'}), Document(page_content=\"Technology[edit]\\nMain article: Ancient Roman technology\\nPont du Gard in France is a Roman aqueduct built in c.\\xa019\\xa0BC. It is a World Heritage Site.\\nAncient Rome boasted impressive technological feats, using many advancements that were lost in the Middle Ages and not rivalled again until the 19th and 20th centuries. An example of this is insulated glazing, which was not invented again until the 1930s. Many practical Roman innovations were adopted from earlier Greek designs. Advancements were often divided and based on craft. Artisans guarded technologies as trade secrets.[223]\\nRoman civil engineering and military engineering constituted a large part of Rome's technological superiority and legacy, and contributed to the construction of hundreds of roads, bridges, aqueducts, public baths, theatres and arenas. Many monuments, such as the Colosseum, Pont du Gard, and Pantheon, remain as testaments to Roman engineering and culture.\", metadata={'language': 'en', 'source': 'https://en.wikipedia.org/wiki/Ancient_Rome', 'title': 'Ancient Rome - Wikipedia'}), Document(page_content=\"Archaeological evidence of settlement around Rome starts to emerge c.\\u20091000\\xa0BC.[4] Large-scale organisation appears only c.\\u2009800\\xa0BC, with the first graves in the Esquiline Hill's necropolis, along with a clay and timber wall on the bottom of the Palatine Hill dating to the middle of the 8th century\\xa0BC. Starting from c.\\u2009650\\xa0BC, the Romans started to drain the valley between the Capitoline and Palatine Hills, where today sits the Roman Forum.[5] By the sixth century, the Romans were constructing the Temple of Jupiter Optimus Maximus on the Capitoline and expanding to the Forum Boarium located between the Capitoline and Aventine Hills.[6]\", metadata={'language': 'en', 'source': 'https://en.wikipedia.org/wiki/Ancient_Rome', 'title': 'Ancient Rome - Wikipedia'})]\n                


Question: What was built in ancient Rome?"
  ]
 ```
    

In [9]:
# from langchain.globals import set_debug
# set_debug(True)
# rag_chain.invoke("What was built in ancient Rome?")

<a id='lab'></a>
# Q&A Lab
We will use the meetings notes stored in `my_docs`. These notes were generated by GPT, and will be loaded, split, embedded, and stored as before. We will then pass our docs along with our question to the language model.

**Objectives:**
1. Load `my_documents` from file folder, split, embed, and store in a Chroma vector database.
1. Build a RAG chain that searches for docs based on a similarity search and returns relevant docs, then passes your retrevied docs as context along with a user prompt to an LLM, and returns the result. 
1. Bonus! Compare result to a basic similarity search. 

In [10]:
# set_debug(False)

In [12]:
# load 
notes_loader = DirectoryLoader('assets/my_docs')
meeting_notes = notes_loader.load()

#split 
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=20,
    length_function=len,
)
split_notes = text_splitter.split_documents(meeting_notes)

# create a vector database called db_notes
db_notes = Chroma.from_documents(split_notes, OpenAIEmbeddings(openai_api_key="...", model="text-embedding-3-large"))

In [13]:
# Retrieve 
notes_retriever = db_notes.as_retriever(search_type="similarity")

In [14]:
# Generate with RAG Chain  

# Specify Model 
model = ChatOpenAI(openai_api_key="...", model="gpt-4-0125-preview")

# Write Prompt
prompt_txt= """Answer the question based only on the following context: {context}
                Question: {question}
            """
notes_prompt = ChatPromptTemplate.from_template(prompt_txt)

# Create RAG Chain 
chain = ( {"context": notes_retriever, "question": RunnablePassthrough()}
    | notes_prompt
    | model
    | StrOutputParser()
)

# invoke chain with the user question (feel free to edit the question! )
chain.invoke("What was our revenue discussion?")

"Based on the documents provided, the revenue discussion took place during the meeting on November 30, 2021. In this meeting, there was a brainstorming of ideas for new revenue streams and market expansion as part of the discussion on improving the company's financial performance and profitability."

### Bonus: Compare

Notice the difference in the retrieved docs and user question being passed to a language model (above) vs. a basic similarity search (below) 

In [15]:
#similarity search comparison 
# Similarity Search Query 
query = "What was discussed about revenue?"
notes_docs= db_notes.similarity_search(query)
print(notes_docs[0].page_content)

Meeting Notes - November 30, 2021

Reviewed financial reports and analyzed the company's performance

Discussed cost-cutting measures to improve profitability

Brainstormed ideas for new revenue streams and market expansion

Discussed employee satisfaction and proposed initiatives for boosting morale

Agreed on next steps and scheduled a strategic planning session for the upcoming year.
